# Simple Groq Chatbot with MemorySaver

A standalone chatbot using only Groq API and MemorySaver for conversation memory.

In [16]:
# Install required packages
!pip install groq langgraph langchain-groq tavily

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for tavily: filename=tavily-1.1.0-py3-none-any.whl size=6170 sha256=2953f0fb921a109bc1c7733d51f25600286bd8a5e7085dbd5e0515cdf9f5d3be
  Stored in directory: c:\users\abhay\appdata\local\pip\cache\wheels\51\08\fd\a42b1feea25355e9e6357061510b277422f42ee1f044aa24d3
Successfully built tavily


In [17]:
import os
from groq import Groq
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from typing import TypedDict, List
import getpass

In [18]:
# Set your Groq API key
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

# Initialize Groq client
client = Groq(api_key=os.environ["GROQ_API_KEY"])

In [19]:
# Define the state for our chatbot
class State(TypedDict):
    messages: List[HumanMessage | AIMessage | SystemMessage]

memory saver

In [20]:
# Initialize MemorySaver
memory = MemorySaver()

# Create the chatbot function
def chatbot_node(state: State):
    """Process messages and generate response using Groq"""
    messages = state["messages"]
    
    # Convert messages for Groq API
    groq_messages = []
    for msg in messages:
        if isinstance(msg, HumanMessage):
            groq_messages.append({"role": "user", "content": msg.content})
        elif isinstance(msg, AIMessage):
            groq_messages.append({"role": "assistant", "content": msg.content})
        elif isinstance(msg, SystemMessage):
            groq_messages.append({"role": "system", "content": msg.content})
    
    # Get response from Groq
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=groq_messages,
        temperature=0.7,
        max_tokens=1000
    )
    
    # Add AI response to messages
    ai_response = AIMessage(content=response.choices[0].message.content)
    messages.append(ai_response)
    
    return {"messages": messages}

In [21]:
# Build the graph
graph = StateGraph(State)

# Add nodes
graph.add_node("chatbot", chatbot_node)

# Add edges
graph.add_edge(START, "chatbot")
graph.add_edge("chatbot", END)

# Compile with memory
app = graph.compile(checkpointer=memory)

In [22]:
# Initialize conversation with system message
def create_conversation(thread_id: str = "default"):
    """Create or resume a conversation"""
    initial_state = {
        "messages": [
            SystemMessage(content="You are a helpful AI assistant. Remember our conversation and provide contextually relevant responses.")
        ]
    }
    
    config = {"configurable": {"thread_id": thread_id}}
    return app.invoke(initial_state, config)

# Start a conversation
print("🤖 Simple Groq Chatbot with MemorySaver initialized!")
print("Type 'quit' to exit or 'new' to start a new conversation thread.")

🤖 Simple Groq Chatbot with MemorySaver initialized!
Type 'quit' to exit or 'new' to start a new conversation thread.


In [23]:
# Interactive chat function
def chat_interactive():
    thread_id = "default"
    
    # Initialize conversation
    state = create_conversation(thread_id)
    config = {"configurable": {"thread_id": thread_id}}
    
    while True:
        user_input = input("\n👤 You: ").strip()
        
        if user_input.lower() in ['quit', 'exit', 'q']:
            print("🤖 Goodbye!")
            break
        
        if user_input.lower() == 'new':
            thread_id = input("Enter thread ID (or press Enter for default): ").strip() or f"thread_{len(memory.storage)}"
            state = create_conversation(thread_id)
            config = {"configurable": {"thread_id": thread_id}}
            print(f"🤖 Started new conversation thread: {thread_id}")
            continue
        
        if user_input.lower() == 'threads':
            print(f"📝 Active threads: {list(memory.storage.keys())}")
            continue
        
        # Add user message and get response
        user_message = HumanMessage(content=user_input)
        
        # Update state with user message
        current_state = {"messages": state["messages"] + [user_message]}
        
        # Get AI response
        result = app.invoke(current_state, config)
        
        # Update state
        state = result
        
        # Print AI response
        ai_response = result["messages"][-1].content
        print(f"🤖 AI: {ai_response}")

# Start chatting
chat_interactive()

🤖 AI: The Prime Minister of India is a significant position in the country's government. As of my knowledge cutoff, the Prime Minister of India is Narendra Modi. He has been in office since May 26, 2014. Would you like to know more about his policies, achievements, or something else related to the Indian government?
🤖 AI: Hello again! It's nice to continue our conversation. We were just discussing the Prime Minister of India, Narendra Modi. Is there something specific you'd like to know or talk about, or would you like to start a new topic? I'm here to help and chat with you!
🤖 AI: It seems like you didn't type anything. That's okay! Let's try again. Is there something on your mind that you'd like to talk about, or would you like me to suggest some conversation topics? I'm here to listen and help.
🤖 Goodbye!


## Features

### MemorySaver Integration:
- **Automatic Memory**: Conversation history is automatically saved
- **Thread Management**: Multiple conversation threads supported
- **Persistence**: Memory persists across notebook sessions

### Groq AI Integration:
- **Llama 3.3 70B**: Uses Groq's fast Llama model
- **Context Awareness**: Remembers previous messages in the thread
- **Natural Conversation**: Maintains conversational flow

### Commands:
- `quit`/`exit`: End the chat
- `new`: Start a new conversation thread
- `threads`: List all active conversation threads

# 🛠 Enhancing Your Chatbot with Tools for Real-Time Data
#1️⃣ Problem with Static LLMs
•	Your LLM is trained only up to a fixed date.


# 🚀 Complete LangGraph Agent Project

## Building a Smart Chatbot with Tools Integration

This section demonstrates the complete workflow of building a LangGraph agent with:
- State management
- Tool integration  
- Conditional routing
- Memory persistence

In [ ]:
# Complete LangGraph Chatbot with Tools Integration
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from typing import Annotated, Literal
import json

# 1️⃣ Create Enhanced State with Tool Results
class AgentState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage | SystemMessage], "Conversation messages"]
    tool_calls: List[str]  # Track which tools were called
    query_type: str  # Track query category for routing

# 2️⃣ Create Tools List
@tool
def tavily_search_tool(query: str) -> str:
    """Search the web for current information about any topic. Use this for news, current events, or real-time information."""
    try:
        if 'tavily_client' not in globals():
            return "❌ Tavily client not initialized. Please set up Tavily API key first."
        
        response = tavily_client.search(
            query=query,
            max_results=3,
            include_answer=True,
            search_depth="basic"
        )
        
        if response.get('answer'):
            return f"🔍 Search Results: {response['answer']}"
        elif response.get('results'):
            top_result = response['results'][0]
            return f"🔍 Found: {top_result['title']}\n📄 {top_result['content'][:200]}..."
        else:
            return "No search results found."
    except Exception as e:
        return f"Search error: {str(e)}"

@tool
def get_current_time_tool() -> str:
    """Get the current date and time. Use this when user asks about time or date."""
    from datetime import datetime
    now = datetime.now()
    return f"📅 Current time: {now.strftime('%Y-%m-%d %H:%M:%S')}"

@tool
def calculate_tool(expression: str) -> str:
    """Calculate mathematical expressions. Use for basic math operations like addition, subtraction, multiplication, division."""
    try:
        # Safe evaluation of basic math
        allowed_chars = set('0123456789+-*/.() ')
        if not all(c in allowed_chars for c in expression):
            return "❌ Invalid characters in expression. Only use numbers, +, -, *, /, (, )"
        
        result = eval(expression)
        return f"🧮 {expression} = {result}"
    except Exception as e:
        return f"❌ Calculation error: {str(e)}"

# 3️⃣ Create Tools List
tools = [tavily_search_tool, get_current_time_tool, calculate_tool]

# 4️⃣ Initialize LLM and Bind Tools
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.1,  # Lower temperature for better tool calling
    api_key=os.environ["GROQ_API_KEY"]
)

# Bind tools to LLM to create an Agent
llm_with_tools = llm.bind_tools(tools)

# 5️⃣ Create Chatbot Node
def chatbot_node(state: AgentState):
    """Main chatbot node that decides whether to use tools or answer directly."""
    messages = state["messages"]
    
    # Get response from LLM with tools
    response = llm_with_tools.invoke(messages)
    
    # Track if tools were called
    tool_calls = [tool_call["name"] for tool_call in response.tool_calls] if response.tool_calls else []
    
    return {
        "messages": [response],
        "tool_calls": tool_calls,
        "query_type": "tool_call" if tool_calls else "direct_response"
    }

# 6️⃣ Create Tool Node (built-in from LangGraph)
tool_node = ToolNode(tools)

# 7️⃣ Create Conditional Edge for Routing
def route_to_tools(state: AgentState) -> Literal["tools", "chatbot"]:
    """Route to tools if LLM made tool calls, otherwise continue."""
    if state.get("tool_calls"):
        return "tools"
    return "chatbot"

# 8️⃣ Build the Complete Graph
agent_graph = StateGraph(AgentState)

# Add nodes
agent_graph.add_node("chatbot", chatbot_node)
agent_graph.add_node("tools", tool_node)

# Add edges
agent_graph.add_edge(START, "chatbot")
agent_graph.add_conditional_edges(
    "chatbot",
    route_to_tools,
    {"tools": "tools", "chatbot": END}
)
agent_graph.add_edge("tools", "chatbot")

# Compile with memory
agent_app = agent_graph.compile(checkpointer=memory)

print("🤖 Complete LangGraph Agent with Tools ready!")
print(f"🔧 Available tools: {[tool.name for tool in tools]}")
print("📝 The agent will automatically decide when to use tools!")

🤖 Complete LangGraph Agent with Tools ready!
🔧 Available tools: ['tavily_search', 'get_current_time', 'calculate']
📝 The agent will automatically decide when to use tools!


In [ ]:
# Interactive Agent Chat with Tools
def chat_with_agent():
    """Chat with the LangGraph agent that has tool access."""
    thread_id = "agent_thread"
    config = {"configurable": {"thread_id": thread_id}}
    
    # Initialize with system message
    initial_state = {
        "messages": [
            SystemMessage(content="""You are a helpful AI assistant with access to tools. 
            Use tools when appropriate:
            - Use tavily_search_tool for current information, news, or real-time data
            - Use get_current_time_tool when asked about time or date
            - Use calculate_tool for math calculations
            Always respond naturally after using tools. Don't explain that you used tools.""")
        ],
        "tool_calls": [],
        "query_type": "direct_response"
    }
    
    # Start conversation
    try:
        agent_app.invoke(initial_state, config)
        print("✅ Agent initialized successfully!")
    except Exception as e:
        print(f"❌ Error initializing agent: {e}")
        return
    
    print("🤖 LangGraph Agent with Tools ready!")
    print("💡 Try asking me to:")
    print("   - Search for 'latest AI news'")
    print("   - Calculate '25 * 4 + 10'")
    print("   - Get current time")
    print("   - Or just chat normally!")
    print("Type 'quit' to exit.\n")
    
    while True:
        user_input = input("👤 You: ").strip()
        
        if user_input.lower() in ['quit', 'exit', 'q']:
            print("🤖 Goodbye!")
            break
        
        try:
            # Add user message and get response
            user_message = HumanMessage(content=user_input)
            
            # Get current state and add user message
            current_state = {"messages": [user_message]}
            
            # Invoke agent
            result = agent_app.invoke(current_state, config)
            
            # Print AI response
            ai_response = result["messages"][-1].content
            print(f"🤖 Agent: {ai_response}")
            
            # Show tool usage info
            if result.get("tool_calls"):
                print(f"🔧 Tools used: {', '.join(result['tool_calls'])}")
                
        except Exception as e:
            print(f"❌ Error: {e}")
            print("Please try again or check your API keys.")

# Start the agent chat
chat_with_agent()

🤖 LangGraph Agent with Tools ready!
💡 Try asking me to:
   - Search for 'latest AI news'
   - Calculate '25 * 4 + 10'
   - Get current time
   - Or just chat normally!
Type 'quit' to exit.



BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<function=tavily_search{"query": "AI advancements"}</function>'}}

In [ ]:
# Test the Complete Agent Workflow
def test_agent_workflow():
    """Test the agent with different types of queries."""
    thread_id = "test_agent"
    config = {"configurable": {"thread_id": thread_id}}
    
    test_queries = [
        "What is 15 * 8?",
        "Search for latest AI developments",
        "What time is it now?",
        "Tell me a joke",
        "Calculate (100 + 50) / 3"
    ]
    
    # Initialize
    initial_state = {
        "messages": [
            SystemMessage(content="You are a helpful assistant with tools.")
        ],
        "tool_calls": [],
        "query_type": "direct_response"
    }
    
    agent_app.invoke(initial_state, config)
    
    print("🧪 Testing Agent Workflow:\n")
    
    for i, query in enumerate(test_queries, 1):
        print(f"Test {i}: {query}")
        print("-" * 40)
        
        user_message = HumanMessage(content=query)
        result = agent_app.invoke({"messages": [user_message]}, config)
        
        ai_response = result["messages"][-1].content
        print(f"Response: {ai_response}")
        
        if result.get("tool_calls"):
            print(f"Tools used: {result['tool_calls']}")
        else:
            print("Tools used: None (direct response)")
        
        print("\n" + "="*50 + "\n")

# Run the test
test_agent_workflow()

In [24]:
# Tavily Search Integration
from tavily import TavilyClient

# Initialize Tavily client
if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

try:
    tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
    print("✅ Tavily client initialized successfully!")
except Exception as e:
    print(f"❌ Error initializing Tavily client: {e}")
    print("Please make sure you have the correct API key and internet connection.")

def search_web(query: str, max_results: int = 5) -> str:
    """Search the web using Tavily and return formatted results"""
    try:
        if 'tavily_client' not in globals():
            return "❌ Tavily client not initialized. Please check your API key."
            
        response = tavily_client.search(
            query=query,
            max_results=max_results,
            search_depth="basic",
            include_answer=True,
            include_raw_content=False
        )
        
        if not response.get('results'):
            return "No search results found."
        
        search_summary = f"🔍 Search results for '{query}':\n\n"
        
        # Add answer if available
        if response.get('answer'):
            search_summary += f"💡 Answer: {response['answer']}\n\n"
        
        # Add top results
        for i, result in enumerate(response['results'][:max_results], 1):
            search_summary += f"{i}. {result['title']}\n"
            search_summary += f"   📄 {result['content'][:200]}...\n"
            search_summary += f"   🔗 {result['url']}\n\n"
        
        return search_summary
        
    except Exception as e:
        return f"❌ Search error: {str(e)}"

# Test the search function
def test_search():
    """Test Tavily search functionality"""
    try:
        test_queries = [
            "latest AI news 2024",
            "Python programming tips",
            "machine learning tutorials"
        ]
        
        for query in test_queries:
            print(f"\n{'='*50}")
            results = search_web(query, max_results=3)
            print(results)
            print(f"{'='*50}")
    except Exception as e:
        print(f"❌ Test search error: {e}")

print("🔍 Tavily Search integration ready!")
print("Use search_web('your query') to search the web.")
print("Run test_search() to test the functionality.")

✅ Tavily client initialized successfully!
🔍 Tavily Search integration ready!
Use search_web('your query') to search the web.
Run test_search() to test the functionality.


In [29]:
search_web("you have info till which year?")

"🔍 Search results for 'you have info till which year?':\n\n💡 Answer: The Information Age began in the mid-20th century, around the 1970s, and continues to evolve today.\n\n1. Information Age - Simple English Wikipedia, the free encyclopedia\n   📄 The Information Age is a historical period that began in the middle of the 20th century. It is defined by a quick change from older industries,...\n   🔗 https://simple.wikipedia.org/wiki/Information_Age\n\n2. What is the Information Age? - Quora\n   📄 It's the Age when your phone, computer, TV, refrigerator and commode collect data about you, so that companies can sell you more stuff....\n   🔗 https://www.quora.com/What-is-the-Information-Age\n\n3. If You Think We're Still in the Age of Information, Think Again\n   📄 Yes, the information age is still in the process of evolving, and we are, to some degree, still in it. This fact, most people already know....\n   🔗 https://www.linkedin.com/pulse/you-think-were-still-age-information-again-mike-do

In [25]:
# Test memory functionality
def test_memory():
    """Test the MemorySaver functionality"""
    test_thread = "test_thread"
    config = {"configurable": {"thread_id": test_thread}}
    
    # Create test conversation
    initial_state = {
        "messages": [
            SystemMessage(content="You are a helpful assistant."),
            HumanMessage(content="Hello! My name is Abhay Nautiyal MTech Student of CSE GBPIET."),
        ]
    }
    
    # Get first response
    result1 = app.invoke(initial_state, config)
    print("First response:", result1["messages"][-1].content)
    
    # Continue conversation
    follow_up = {
        "messages": result1["messages"] + [
            HumanMessage(content="What's my name?")
        ]
    }
    
    result2 = app.invoke(follow_up, config)
    print("Second response:", result2["messages"][-1].content)
    
    # Check memory storage
    print(f"\nMemory threads: {list(memory.storage.keys())}")
    print(f"Messages in {test_thread}: {len(memory.storage.get(test_thread, {}).get('messages', []))}")

# Run test
test_memory()

First response: Hello Abhay! Nice to meet you. I'm glad you introduced yourself. It's great to know that you're an MTech student of Computer Science and Engineering at GBPIET. How's your experience been so far? Are you enjoying your studies and research? Is there anything specific you'd like to talk about or any help you need? I'm here to assist you.
Second response: Your name is Abhay Nautiyal.

Memory threads: ['default', 'test_thread']
Messages in test_thread: 0
